### imports

In [100]:
import pandas as pd
import numpy as np
from PyPDF2 import PdfReader

### code

In [36]:
# Open the PDF file
pdf_path = '/Users/AmandaLi/Desktop/Courses/STA141B/cleaned-ea-survey.pdf'

In [37]:
reader = PdfReader(pdf_path)

In [38]:
text_by_page = []
for page_num in range(len(reader.pages)):
    page = reader.pages[page_num]
    text = page.extract_text()
    text_by_page.append(text)

In [191]:
def initial_cleanup(df):
    df = df.iloc[7:]
    df = df.iloc[:-3]
    df = df.drop(df.columns[8:], axis=1)
    return df

In [165]:
def str_to_na(df):
    df.replace(None, np.nan, inplace=True)

In [248]:
def shift_row_right(df):
    # Identify rows where the 'Percent of District Pop in County' is None
    row_with_none = df['Percent of District Pop in County'].isna()

    # Loop over the rows that match the condition
    for idx in df[row_with_none].index:
        # Shift all columns from left to right, except the last column (temp)
        for col in range(len(df.columns) - 1, 0, -1):
            current_col = df.columns[col - 1]
            next_col = df.columns[col]
            # Move the value in the current column to the next column
            df.at[idx, next_col] = df.at[idx, current_col]
            # Set the current column to None
            df.at[idx, current_col] = None

        # Set 'County' column to None (as per original logic)
        df.at[idx, 'County'] = None
    
    # Display the updated DataFrame
    return df

In [260]:
def new_hanover_shift(df):
    # Check if the 'County' column in the row is "New"
    for idx, row in df.iterrows():
        if row['County'] == 'New' and df.at[idx, 'District'] == 'Hanover':
            # Combine 'New' and 'Hanover' into the 'County' column
            df.at[idx, 'County'] = 'New Hanover'
            
            # Shift all the values to the left for this specific row
            df.at[idx, 'District'] = df.at[idx, 'Total County Pop']
            df.at[idx, 'Total County Pop'] = df.at[idx, 'Total District Pop']
            df.at[idx, 'Total District Pop'] = df.at[idx, 'County Pop in District']
            df.at[idx, 'County Pop in District'] = df.at[idx, 'Percent of County Pop in District']
            df.at[idx, 'Percent of County Pop in District'] = df.at[idx, 'Percent of District Pop in County']
            df.at[idx, 'Percent of District Pop in County'] = None  # Set the last column to None
            df.at[idx, 'temp'] = None  # Set the last column ('temp') to None

    return df

In [192]:
def rename_cols(table):
    table.columns = ["County", 
                     "District", 
                     "Total County Pop", 
                     "Total District Pop",
                     "County Pop in District", 
                     "Percent of County Pop in District", 
                     "Percent of District Pop in County",
                    "temp"]
    return table

In [263]:
def generate_df(text): 
    lines = text.split('\n')  # Split the text into lines
    data = [line.split() for line in lines]
    
    df = pd.DataFrame(data)
    df = initial_cleanup(df)
    df = rename_cols(df)
    df = shift_row_right(df)
    
    # Print the DataFrame to inspect the result
    return df

In [264]:
table0 = generate_df(text_by_page[0])
table1 = generate_df(text_by_page[1])
table2 = generate_df(text_by_page[2])

In [267]:
table1 = new_hanover_shift(table1)

In [305]:
def reload_tables(table0, table1, table2):
    tables = [table0, table1, table2]
    return tables

In [307]:
cleaned_tables = reload_tables(table0, table1, table2)
combined_df = pd.concat(cleaned_tables, ignore_index=True)
# drop temp column with None
combined_df = combined_df.drop(columns ="temp", axis = 1)

In [308]:
# drop last row
combined_df = combined_df[:-1]

In [309]:
combined_df

,County,District,Total County Pop,Total District Pop,County Pop in District,Percent of County Pop in District,Percent of District Pop in County
0,Alamance,13,"151,131","733,498","151,131",100.00%,20.60%
1,Alexander,5,"37,198","733,499","37,198",100.00%,5.07%
2,Alleghany,5,"11,155","733,499","11,155",100.00%,1.52%
3,Anson,9,"26,948","733,499","26,948",100.00%,3.67%
4,Ashe,5,"27,281","733,499","27,281",100.00%,3.72%
...,...,...,...,...,...,...,...
107,Wayne,1,"122,623","733,498","122,623",100.00%,16.72%
108,Wilkes,5,"69,340","733,499","69,340",100.00%,9.45%
109,Wilson,1,"81,234","733,498","81,234",100.00%,11.07%
110,Yadkin,10,"38,406","733,499","38,406",100.00%,5.24%


In [310]:
# changing column types for easy data analysis
combined_df['District'] = combined_df['District'].astype(float)
combined_df['Total County Pop'] = combined_df['Total County Pop'].str.replace(',', '').astype(int)
combined_df['Total District Pop'] = combined_df['Total District Pop'].str.replace(',', '').astype(int)
combined_df['County Pop in District'] = combined_df['County Pop in District'].str.replace(',', '').astype(int)

In [311]:
combined_df['Percent of County Pop in District'] = (combined_df['County Pop in District'] / combined_df['Total County Pop']) * 100
combined_df['Percent of District Pop in County'] = (combined_df['County Pop in District'] / combined_df['Total District Pop']) * 100

In [317]:
combined_df['County'] = combined_df['County'].fillna(method='ffill')

/var/folders/xd/ljd80glj4631l10fr9j779bm0000gn/T/ipykernel_46423/4216097951.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df['County'] = combined_df['County'].fillna(method='ffill')


In [319]:
# create dictionary
combined_dict = {}

# populate dictionary
for index, row in combined_df.iterrows():
    county = row['County']
    
    # for current row
    district_info = {
        'District': row['District'],
        'Total County Pop': row['Total County Pop'],
        'Total District Pop': row['Total District Pop'],
        'County Pop in District': row['County Pop in District'],
        'Percent of County Pop in District': row['Percent of County Pop in District'],
        'Percent of District Pop in County': row['Percent of District Pop in County']
    }
    
    # if county is already a key, append info
    if county in combined_dict:
        combined_dict[county].append(district_info)
    else:
        # if not already a key, create new key
        combined_dict[county] = [district_info]

### write csv and json to local machine

In [322]:
combined_df.to_csv('nc_pop.csv', index=False) 

In [323]:
import json
with open('nc_pop.json', 'w') as fp:
    json.dump(combined_dict, fp)